In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import collections

In [2]:
# Copyright (c) 2017-2019 The University of Manchester
#
# This program is free software: you can redistribute it and/or modify
# it under the terms of the GNU General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.
#
# This program is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# GNU General Public License for more details.
#
# You should have received a copy of the GNU General Public License
# along with this program.  If not, see <http://www.gnu.org/licenses/>.
from collections import OrderedDict
from enum import Enum
import struct

from spinn_front_end_common.abstract_models import AbstractProvidesNKeysForPartition
from spinn_utilities.overrides import overrides
from pacman.executor.injection_decorator import inject_items
from pacman.model.graphs.machine import MachineVertex
from pacman.model.resources import ResourceContainer, VariableSDRAM
from pacman.utilities.utility_calls import is_single
from spinn_front_end_common.utilities.constants import (
    SYSTEM_BYTES_REQUIREMENT, BYTES_PER_WORD)
from spinn_front_end_common.utilities.exceptions import ConfigurationException
from spinn_front_end_common.utilities.helpful_functions import (
    locate_memory_region_for_placement)
from spinn_front_end_common.abstract_models.impl import (
    MachineDataSpecableVertex)
from spinn_front_end_common.interface.buffer_management.buffer_models import (
    AbstractReceiveBuffersToHost)
from spinn_front_end_common.interface.buffer_management import (
    recording_utilities)
from spinnaker_graph_front_end.utilities import SimulatorVertex
from spinnaker_graph_front_end.utilities.data_utils import (
    generate_system_data_region)

from data_specification.enums.data_type import DataType

from spinn_front_end_common.utilities import exceptions
import logging

logger = logging.getLogger(__name__)


class LatticeBasicCell(
    SimulatorVertex, MachineDataSpecableVertex,
    AbstractReceiveBuffersToHost,
    AbstractProvidesNKeysForPartition
):
    """ A Boltzmann Lattice which represents a cell within the 2d fabric
    """
    
    PARTITION_ID = "STATE"

    TRANSMISSION_DATA_SIZE = 2 * BYTES_PER_WORD  # has key and key
    STATE_DATA_SIZE = BYTES_PER_WORD  # 1 or 2 based off dead or alive
    # alive states, dead states
    NEIGHBOUR_INITIAL_STATES_SIZE = 2 * BYTES_PER_WORD
    RECORDING_ELEMENT_SIZE = STATE_DATA_SIZE  # A recording of the volocity

    POSITION_DATA_SIZE = 2 * BYTES_PER_WORD # x position and y position

    NEIGHBOUR_KEYS_SIZE = 9 * BYTES_PER_WORD  # for 8 directions and a mask

    VELOCITY_SIZE = 2 * BYTES_PER_WORD  # u_x and u_y

    VERTEX_INDEX_SIZE = BYTES_PER_WORD

    # The order of which directions are writeen to sdram
    ORDER_OF_DIRECTIONS = ["N", "W", "S", "E", "NW", "SW", "SE", "NE"]
    # Regions for populations
    DATA_REGIONS = Enum(
        value="DATA_REGIONS",
        names=[('SYSTEM', 0),
               ('TRANSMISSIONS', 1),
               ('POSITION', 2),
               ('NEIGHBOUR_KEYS', 3),
               ('VELOCITY', 4),
               ('VERTEX_INDEX', 5),
               ('RESULTS', 6)])

    def __init__(self, label, x_position, y_position, u_x, u_y):
        super(LatticeBasicCell, self).__init__(label, "lattice_cell.aplx")
        AbstractProvidesNKeysForPartition.__init__(self)
        # app specific data items
        self._x_position = x_position
        self._y_position = y_position
        self.u_x = u_x
        self.u_y = u_y

        self._loccation_vertices = OrderedDict()
        for direction in self.ORDER_OF_DIRECTIONS:
            self._loccation_vertices[direction] = None

    def set_direction_vertex(self, direction, vertex):
        """
        Add a vertex to the corresponding direction
        """
        self._loccation_vertices[direction] = vertex

    def _write_key_data(self, spec, routing_info, graph):
        """
        Write the keys of its 8 neighbours
        """
        spec.switch_write_focus(region=self.DATA_REGIONS.NEIGHBOUR_KEYS.value)
        incoming_edges = graph.get_edges_ending_at_vertex_with_partition_name(self, self.PARTITION_ID)

        # verify the number of edges
        if len(incoming_edges) != 8:
            print(incoming_edges)
            raise exceptions.ConfigurationException("Should only have 8 edges")

        # get incoming edges
        for direction in self._loccation_vertices:
            key = routing_info.get_routing_info_from_pre_vertex(
                self._loccation_vertices[direction],
                self.PARTITION_ID).keys_and_masks[0].key
            if key is not None:
                spec.write_value(data=key)
            else:
                logger.warning("This lattice miss a edge from direction {}".format(direction))
                spec.write_value(data_type=DataType.INT32, data=-1)

        mask = routing_info.get_routing_info_from_pre_vertex(self._loccation_vertices["S"],
                                                             self.PARTITION_ID).keys_and_masks[0].mask
        spec.write_value(data=mask, data_type=DataType.UINT32)

    @inject_items({"data_n_time_steps": "DataNTimeSteps"})
    @overrides(
        MachineDataSpecableVertex.generate_machine_data_specification,
        additional_arguments={"data_n_time_steps"})
    def generate_machine_data_specification(
            self, spec, placement, machine_graph, routing_info, iptags,
            reverse_iptags, machine_time_step, time_scale_factor,
            data_n_time_steps):
        # Generate the system data region for simulation .c requirements
        generate_system_data_region(spec, self.DATA_REGIONS.SYSTEM.value,
                                    self, machine_time_step, time_scale_factor)

        # reserve memory regions for every data region
        spec.reserve_memory_region(
            region=self.DATA_REGIONS.TRANSMISSIONS.value,
            size=self.TRANSMISSION_DATA_SIZE, label="inputs")

        spec.reserve_memory_region(
            region=self.DATA_REGIONS.POSITION.value,
            size=self.POSITION_DATA_SIZE, label="position"
        )

        spec.reserve_memory_region(
            region=self.DATA_REGIONS.NEIGHBOUR_KEYS.value,
            size=self.NEIGHBOUR_KEYS_SIZE
        )

        spec.reserve_memory_region(
            region=self.DATA_REGIONS.VELOCITY.value,
            size=self.VELOCITY_SIZE
        )

        spec.reserve_memory_region(
            region=self.DATA_REGIONS.VERTEX_INDEX.value,
            size=self.VERTEX_INDEX_SIZE
        )

        spec.reserve_memory_region(
            region=self.DATA_REGIONS.RESULTS.value,
            size=recording_utilities.get_recording_header_size(1))

        # get recorded buffered regions sorted
        spec.switch_write_focus(self.DATA_REGIONS.RESULTS.value)
        spec.write_array(recording_utilities.get_recording_header_array(
            [self.RECORDING_ELEMENT_SIZE * data_n_time_steps]))

        # check got right number of keys and edges going into me
        partitions = \
            machine_graph.get_outgoing_edge_partitions_starting_at_vertex(self)
        if not is_single(partitions):
            raise ConfigurationException(
                "Can only handle one type of partition.")

        # check for duplicates
        edges = list(machine_graph.get_edges_ending_at_vertex(self))
        if len(edges) != 8:
            raise ConfigurationException(
                "I've not got the right number of connections. I have {} "
                "instead of 8".format(
                    len(machine_graph.get_edges_ending_at_vertex(self))))

        for edge in edges:
            if edge.pre_vertex == self:
                raise ConfigurationException(
                    "I'm connected to myself, this is deemed an error"
                    " please fix.")

        # write key needed to transmit with
        key = routing_info.get_first_key_from_pre_vertex(
            self, self.PARTITION_ID)

        spec.switch_write_focus(
            region=self.DATA_REGIONS.TRANSMISSIONS.value)
        spec.write_value(0 if key is None else 1)
        spec.write_value(0 if key is None else key)

        # write POSITION data
        spec.switch_write_focus(
            region=self.DATA_REGIONS.POSITION.value
        )
        spec.write_value(int(self._x_position))
        spec.write_value(int(self._y_position))

        #write VERTEX_INDEX data. Mainly for add a random delay
        spec.switch_write_focus(region=self.DATA_REGIONS.VERTEX_INDEX.value)
        spec.write_value(machine_graph.vertices.index(self))

        # write the neighbour keys and masks
        self._write_key_data(spec, routing_info, machine_graph)

        #write velocity data in two dimension, x and y
        spec.switch_write_focus(region=self.DATA_REGIONS.VELOCITY.value)
        spec.write_value(self.u_x, data_type=DataType.FLOAT_32)
        spec.write_value(self.u_y, data_type=DataType.FLOAT_32)

        # End-of-Spec:
        spec.end_specification()

    def get_data(self, buffer_manager, placement):
        # for buffering output info is taken form the buffer manager
        # get raw data, convert to list of booleans
        raw_data, data_missing = buffer_manager.get_data_by_placement(
            placement, 0)

        # do check for missing data
        if data_missing:
            print("missing_data from ({}, {}, {}); ".format(
                placement.x, placement.y, placement.p))

        # return the data, converted to list of booleans
        return [
            element
            for element in struct.unpack(
                "<{}f".format(len(raw_data) // BYTES_PER_WORD), raw_data)]

    @property
    @overrides(MachineVertex.resources_required)
    def resources_required(self):
        """
        Reserve resources for data regions
        """
        fixed_sdram = (SYSTEM_BYTES_REQUIREMENT + self.TRANSMISSION_DATA_SIZE +
                       self.POSITION_DATA_SIZE +
                       self.NEIGHBOUR_KEYS_SIZE +
                       self.VELOCITY_SIZE +
                       self.VERTEX_INDEX_SIZE +
                       recording_utilities.get_recording_header_size(1) +
                       recording_utilities.get_recording_data_constant_size(1))
        per_timestep_sdram = self.RECORDING_ELEMENT_SIZE
        return ResourceContainer(
            sdram=VariableSDRAM(fixed_sdram, per_timestep_sdram))

    @property
    def x_position(self):
        return self._x_position

    @overrides(AbstractProvidesNKeysForPartition.get_n_keys_for_partition)
    def get_n_keys_for_partition(self, partition, graph_mapper):
        """
        Ask for 8 keys for every lattice for the transmission of 8 fi
        """
        return 8  # for its 8 neighbours to send

    @property
    def y_position(self):
        return self._y_position

    def __repr__(self):
        return self.label

    @overrides(AbstractReceiveBuffersToHost.get_recorded_region_ids)
    def get_recorded_region_ids(self):
        return [0]

    @overrides(AbstractReceiveBuffersToHost.get_recording_region_base_address)
    def get_recording_region_base_address(self, txrx, placement):
        return locate_memory_region_for_placement(
            placement, self.DATA_REGIONS.RESULTS.value, txrx)


In [3]:
from pacman.model.graphs.machine import MachineEdge


class LatticeEdge(MachineEdge):
    """
    Used for conjunction with a lattice
    """

    def __init__(self, pre_vertex, post_vertex, compass, label=None):
        MachineEdge.__init__(
            self, pre_vertex, post_vertex, label=label)
        self._compass = compass

    @property
    def compass(self):
        return self._compass

    def __str__(self):
        return self.__repr__()

    def __repr__(self):
        return "LatticeEdge: {}:{}".format(self._compass, self._label)

In [4]:
# Copyright (c) 2017-2019 The University of Manchester
#
# This program is free software: you can redistribute it and/or modify
# it under the terms of the GNU General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.
#
# This program is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# GNU General Public License for more details.
#
# You should have received a copy of the GNU General Public License
# along with this program.  If not, see <http://www.gnu.org/licenses/>.
import math
import os
# from pacman.model.graphs.machine import MachineEdge

import spinnaker_graph_front_end as front_end

# 12000
runtime = 120000
time_step = 10000
MAX_X_SIZE_OF_FABRIC = 128
MAX_Y_SIZE_OF_FABRIC = 128
n_chips = (MAX_X_SIZE_OF_FABRIC * MAX_Y_SIZE_OF_FABRIC) // 10

ex = [0, 1, 0, -1, 0, 1, -1, -1, 1]
ey = [0, 0, 1, 0, -1, 1, 1, -1, -1]



def initVelocity(x_pos, y_pos):
    """
    Init the velocity u, v = (u_x, u_y)
  
    ydim = xdim = N = 128
    K = 30 / N
    delta = 0.05
  
    u = tanh( K (y - 0.25 * ydim) ) for y <= 0.5 * ydim 
    u = U_0 tanh( K (0.75 * ydim - y) ) for y >  0.5 * ydim
    v = delta sin( 2pi *x / xdim )
    """
    U_0 = 0.01
    K = 30.0
    delta = 0.05
    x_temp = 1.0 * (x_pos) / MAX_X_SIZE_OF_FABRIC
    y_temp = 1.0 * (y_pos) / MAX_Y_SIZE_OF_FABRIC
    if y_temp <= 0.5:
        u_x = U_0 * math.tanh(K * (y_temp - 0.25))
    else:
        u_x = U_0 * math.tanh(K * (0.75 - y_temp))
    u_y = U_0 * delta * math.sin(2 * math.pi * (x_temp + 0.25))
    return u_x, u_y


# set up the front end and ask for the detected machines dimensions
front_end.setup(
    n_chips_required=n_chips, model_binary_folder=os.path.dirname(os.path.abspath("__file__")), machine_time_step=time_step,time_scale_factor=10)

# figure out if machine can handle simulation
cores = front_end.get_number_of_available_cores_on_machine()

# chech if there is enough cores
if cores <= (MAX_X_SIZE_OF_FABRIC * MAX_Y_SIZE_OF_FABRIC):
    raise KeyError("Don't have enough cores to run simulation")

# contain the vertices for the connection aspect
vertices = [
    [None for _ in range(MAX_X_SIZE_OF_FABRIC)]
    for _ in range(MAX_Y_SIZE_OF_FABRIC)]

# build vertices
for x in range(0, MAX_X_SIZE_OF_FABRIC):
    for y in range(0, MAX_Y_SIZE_OF_FABRIC):
        u_x, u_y = initVelocity(x, y)
        vert = LatticeBasicCell(
            "cell{}".format((x * MAX_X_SIZE_OF_FABRIC) + y),
            x, y, u_x, u_y)
        vertices[x][y] = vert
        front_end.add_machine_vertex_instance(vert)

# build edges
for x in range(0, MAX_X_SIZE_OF_FABRIC):
    for y in range(0, MAX_Y_SIZE_OF_FABRIC):
        #   diraction = ["me", "n", "w", "s", "e", "nw", "sw", "se", "ne"]
        positions = [
            (x, (y + 1) % MAX_Y_SIZE_OF_FABRIC, "E"),
            ((x + 1) % MAX_X_SIZE_OF_FABRIC,
             (y + 1) % MAX_Y_SIZE_OF_FABRIC, "SE"),
            ((x + 1) % MAX_X_SIZE_OF_FABRIC, y, "S"),
            ((x + 1) % MAX_X_SIZE_OF_FABRIC,
             (y - 1) % MAX_Y_SIZE_OF_FABRIC, "SW"),
            (x, (y - 1) % MAX_Y_SIZE_OF_FABRIC, "W"),
            ((x - 1) % MAX_X_SIZE_OF_FABRIC,
             (y - 1) % MAX_Y_SIZE_OF_FABRIC, "NW"),
            ((x - 1) % MAX_X_SIZE_OF_FABRIC, y, "N"),
            ((x - 1) % MAX_X_SIZE_OF_FABRIC,
             (y + 1) % MAX_Y_SIZE_OF_FABRIC, "NE")]

         # build edges for each direction for this vertex
        for (dest_x, dest_y, compass) in positions:
            front_end.add_machine_edge_instance(LatticeEdge( vertices[x][y], vertices[dest_x][dest_y],compass, "edge between {} and {}".format(vertices[x][y], vertices[dest_x][dest_y])), LatticeBasicCell.PARTITION_ID)
            vertices[x][y].set_direction_vertex(direction=compass, vertex=vertices[dest_x][dest_y])

# run the simulation
front_end.run(runtime)

# get recorded data
recorded_data = dict()

# if not front_end.use_virtual_machine():
buffer_manager = front_end.buffer_manager()

# get the data per vertex
for x in range(0, MAX_X_SIZE_OF_FABRIC):
    for y in range(0, MAX_Y_SIZE_OF_FABRIC):
        recorded_data[x, y] = vertices[x][y].get_data(
            front_end.buffer_manager(),
            front_end.placements().get_placement_of_vertex(
                vertices[x][y]))

# clear the machine
front_end.stop()


2020-07-19 17:05:19 INFO: Read cfg files: /home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spinn_front_end_common/interface/spinnaker.cfg, /home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spinnaker_graph_front_end/spiNNakerGraphFrontEnd.cfg, /home/spinnaker/.spiNNakerGraphFrontEnd.cfg
2020-07-19 17:05:19 INFO: Will search these locations for binaries: /home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spinn_front_end_common/common_model_binaries : /home/spinnaker
2020-07-19 17:05:19 WARNING: /home/spinnaker/reports has 45 old reports that have not been closed
2020-07-19 17:05:19 WARNING: /home/spinnaker/application_generated_data_files has 141 old reports that have not been closed
2020-07-19 17:05:19 INFO: Setting time scale factor to 10.
2020-07-19 17:05:19 INFO: Setting machine time step to 10000 micro-seconds.
Created spalloc job 5335937
2020-07-19 17:05:19 INFO: Created spalloc job 5335937
Waiting for board power commands to complete.
2020-07-19 17:05:19 INFO: Waiting 

['/home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spinn_front_end_common/interface/spinnaker.cfg', '/home/spinnaker/sPyNNaker/lib/python3.6/site-packages/spinnaker_graph_front_end/spiNNakerGraphFrontEnd.cfg', '/home/spinnaker/.spiNNakerGraphFrontEnd.cfg']


2020-07-19 17:05:29 INFO: Time 0:00:10.077551 taken by SpallocAllocator
2020-07-19 17:05:29 INFO: Creating transceiver for 10.11.223.17
2020-07-19 17:05:29 INFO: Working out if machine is booted
2020-07-19 17:05:33 INFO: Attempting to boot machine
2020-07-19 17:05:48 INFO: Found board with version [Version: SC&MP 3.2.5 at SpiNNaker:0:0:0 (built Thu Aug  1 08:15:06 2019)]
2020-07-19 17:05:48 INFO: Machine communication successful
2020-07-19 17:05:48 WARNING: Link 2630 points to Chip (27, 3) but that is not included 
2020-07-19 17:05:48 WARNING: Link 26311 points to Chip (27, 32) but that is not included 
2020-07-19 17:05:48 WARNING: Link 26320 points to Chip (27, 32) but that is not included 
2020-07-19 17:05:48 WARNING: Link 2722 points to Chip (27, 3) but that is not included 
2020-07-19 17:05:48 WARNING: Link 2833 points to Chip (27, 3) but that is not included 
2020-07-19 17:05:48 WARNING: Link 2844 points to Chip (27, 3) but that is not included 
2020-07-19 17:05:48 WARNING: Link 2

2020-07-19 17:07:47 INFO: Time 0:00:02.683731 taken by WriteMemoryIOData
Executing data specifications and loading data for application vertices
|0%                          50%                         100%|
2020-07-19 17:09:24 INFO: Time 0:01:37.585640 taken by HostExecuteApplicationDataSpecification
Writing fixed route report
|0%                          50%                         100%|
2020-07-19 17:09:26 INFO: Time 0:00:01.293366 taken by FixedRouteFromMachineReport
Loading executables onto the machine
|0%                          50%                         100%|
2020-07-19 17:09:31 INFO: Time 0:00:04.937282 taken by LoadApplicationExecutableImages
Reading Routing Tables from Machine
|0%                          50%                         100%|
2020-07-19 17:10:03 INFO: Time 0:00:31.874302 taken by RoutingTableFromMachineReport
Generating compressed router table report
|0%                          50%                         100%|
2020-07-19 17:10:05 INFO: Time 0:00:01.991916 ta

In [5]:
len(recorded_data[1,1])

24000

In [6]:
# visualise it in text form (bad but no vis this time)
u_x_output = collections.defaultdict(dict)
u_y_output = collections.defaultdict(dict)
for i in range(0, MAX_X_SIZE_OF_FABRIC):
    for j in range(0, MAX_Y_SIZE_OF_FABRIC):
        u_x_output[i][j] = recorded_data[i, j][23998]
        u_y_output[i][j] = recorded_data[i, j][23999]

In [7]:
# calculate the voriticity
voriticity = collections.defaultdict(dict)

for i in range(0, MAX_X_SIZE_OF_FABRIC):
    for j in range(0, MAX_Y_SIZE_OF_FABRIC): 
        voriticity[i][j] = 0.5 * (u_y_output[(i + 1) % MAX_X_SIZE_OF_FABRIC][j] - u_y_output[(i - 1)%MAX_X_SIZE_OF_FABRIC][j]) + 0.5 * (u_x_output[i][(j + 1)%MAX_Y_SIZE_OF_FABRIC] - u_x_output[i][(j - 1) % MAX_Y_SIZE_OF_FABRIC])

voriticity

defaultdict(dict,
            {0: {0: 0.00018925778567790985,
              1: 0.00020406651310622692,
              2: 0.00021735276095569134,
              3: 0.00023136241361498833,
              4: 0.00024407869204878807,
              5: 0.0002578382845968008,
              6: 0.0002722679637372494,
              7: 0.0002866736613214016,
              8: 0.00030100555159151554,
              9: 0.00031441403552889824,
              10: 0.00032719317823648453,
              11: 0.0003416198305785656,
              12: 0.00035850564017891884,
              13: 0.00037347571924328804,
              14: 0.0003882341552525759,
              15: 0.00040425523184239864,
              16: 0.00041801738552749157,
              17: 0.0004346694331616163,
              18: 0.0004501990042626858,
              19: 0.0004642221610993147,
              20: 0.0004777815192937851,
              21: 0.0004919830244034529,
              22: 0.000506159383803606,
              23: 0.000519334804266

In [8]:
# Visulize the voriticity with contour
data = pd.DataFrame.from_dict(voriticity).T
X = np.linspace(0,127,128)
Y = np.linspace(0,127,128)
def f(X,Y):
    return (data.iloc[X,Y])

fig = plt.figure()
plt.title('128 * 128 contour of Vorticity')
ax = plt.subplot(1,1,1)
plt.contour(X, Y, f(X,Y),127, colors='k', linewidths=(0.7,))
plt.contourf(X, Y, f(X,Y),127, alpha=0.25, colors='black', linewidths=0.7)
ax.set_aspect('equal')
plt.tight_layout()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/spinnaker/sPyNNaker/lib/python3.6/site-packages/ipykernel_launcher.py:10: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  # Remove the CWD from sys.path while we load stuff.
/home/spinnaker/sPyNNaker/lib/python3.6/site-packages/ipykernel_launcher.py:12: UserWarning: linewidths is ignored by contourf
  if sys.path[0] == '':


In [26]:
data.to_csv(r'Vorticity_128*128.csv', index = False)